## Adding Error Bounds in Python

This notebook provides instructions for adding error bounds to a fitted curve.  We did this in an earlier notebook (when we calibrated the Zahn cup), but consolidates some of that information in a single notebook.

### Importing the Data

Let's say you have a set of data in a spreadsheet (we'll use Excel here).   The data might look like this:

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/Example_spreadsheet_data.PNG width = 200>
</center>

<br>

First, we want to import this data:

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Change the `home` variable to the location of the data file on your Google drive.  If you changed the name of the file, you'll need to change `filename` as well:

In [ ]:
import pandas as pd
import numpy as np

home = '/gdrive/MyDrive/Teaching/Engr_290/Notebooks_290/Labs_and_Sensors/Arrhenius_Curve/'
filename = 'spreadsheet_test_data'
file = home + filename + ".xlsx"
vis_data = pd.read_excel(file)

The data is stored as a DataFrame called `vis_data`.  For ease of use, we'll change the titles of the columns, and put the data for water into a Series.

In [ ]:
vis_data.columns = ['water_temp','water_vis']
water = pd.Series(data=vis_data.water_vis.values,index=vis_data.water_temp.values)
water.plot(style='o', ylabel='Viscosity (cP)', xlabel='Temperature (C)');

### Defining the Arrhenius curve

Next we need to fit a line.  Just as we did when we calibrated the Zahn cup, we'll set up a function of the form we want (an Arrhenius curve) and then use a least squares algorithm to find the best coefficients for the fit.

<br>

Remember that we could adapt this for any type of curve: if we wanted to fit a quadratic or linear curve, we could simply replace the `arrhenius()` function with a different function!

In [ ]:
def arrhenius(mu_0,B,T_array):
    # Choose the best way to calculate the viscosity value
    mu = mu_0 * np.exp(B/(T_array+273.15))

    # Choose the appropriate return line
    return pd.Series(index = T_array,data=mu)


Remember that to use least squares, we need to define an error function: a function that finds the difference between the curve and the data at each point (this is written as $y_i - y_c$ in our class notes).

In [ ]:
def error_func(params, data):
    arrh = arrhenius(params[0],params[1],data.index)
    errors = arrh - data
    return errors

And now we minimize the error function with `leastsq()` and a first guess at the parameters:

In [ ]:
import scipy.optimize as spo
params = [0.01,1300]
best_params, fit_details = spo.leastsq(error_func, params, water)


Finally, we can plot the fitted curve with the data points:

In [ ]:
arrh_best = arrhenius(best_params[0],best_params[1],water.index)
water.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of Water', style = 'o', label='Data',legend=True);
arrh_best.plot(label='Fitted Arrhenius', legend =True);

### Plotting with error bars

To find the uncertainty in the fitted curve, you will need to find the standard error of the fit (see step 4 of the Zahn cup calibration process).  

<br>  

Once you have the uncertainty, we want to draw a "bounds" around the fit.  This is because the standard error of the fit does *not* tell us the error in each data point, but instead tells us the potential error in the fit: how far off the fit might be.  

<br>

So we want to draw our error bars not from the data points, but from the curve itself. We can do that using MatPlotLib, which is a powerful tool for plotting in Python (and in fact is the code on which our Series and DataFrame plot() functions are built).  `fill_between` defines an upper and lower bound for the curve, and shades in the region.  `alpha` defines the transparency of the shaded region.

<br>

We'll set our uncertainty here to an arbitrary value of $0.02 ~cP$:



In [ ]:
import matplotlib.pyplot as plt
unc = 0.02
water.plot(ylabel = 'viscosity (cP)', xlabel = 'Temperature (C)',
           title = 'Viscosity of Water', style = 'o', label='Data',legend=True);
arrh_best.plot(label='Fitted Arrhenius', legend =True);
plt.fill_between(arrh_best.index, arrh_best.values - unc, arrh_best.values + unc,
                 color = 'gray', alpha = 0.3);

So now that you have a model for this type of plot, you can use it to display your own viscosity data.  Yippee!